# Feature engineering

In this notebook we cover examples of common feature engineering tasks on both numeric and categorical data. The goal isn't to be exhaustive, but provide you with enough examples that you can get the picture.

We begin with numeric features. Let's begin by reading some data.

In [ ]:
import pandas as pd
import numpy as np

auto = pd.read_csv('data/auto-mpg.csv', sep = '\s+', header = None, 
                   names = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 
                            'acceleration', 'model year', 'origin', 'car_name'])
auto.head()

Let's check the data types:

In [ ]:
auto.dtypes

We can see that we have quite a few numeric columns in here. However, we also need to be careful: a column like `cylinders` looks numeric, but probably should be treated as a categorical column. Why? As yourself which one of these two is more likely:

- do any kind of math with `cylinders`, such as taking the log of it, adding it to another column, or being interested in knowing the average number of cylinders?
- use cylinder as a grouping variable, such as wanting to see the average horsepower **by cylinder**?

We can probably agree that the second case is more likely. For this reason, we need convert cylinder into a categorical column. More on categorical columns in the next lesson.

In [ ]:
auto['cylinders'] = auto['cylinders'].astype('category')

Let's now look at some distributions for our numeric columns.

In [ ]:
import seaborn as sns

sns.pairplot(auto);

None of the feature seem to have any extreme values. Let's check for missing values. To see if any columns in the data are missing, we can use the `isnull` method, followed by `any` where the `axis = 0` is used to ask if **any** of values **across columns** is null: `False` means that none is missing, and `True` means that at least one value is missing.

In [ ]:
auto.isnull().any(axis = 0)

Even better would be to use `sum` to get the count of rows in for each column that are missing.

In [ ]:
auto.isnull().sum(axis = 0)

Note that all of this would work only if the missing data is properly marked as null. Often when reading external data sources, other conventions are used to mark data as missing, such as using `-999` for numeric data, or the string `NA` or `?` or the empty string for categorical data. Such values by default would not be picked up by `isnull`. So we would need to first recode them (or overwrite them) with `np.nan`.

### Exercise

Let's illustrate the above point with an example, let's intentially introduce a few missing values in the data, but using characters that won't be recognized as such.

In [ ]:
auto.loc[2, ['mpg', 'car_name']] = [-999, ""] # make mpg and car_name in the 3st row NA
auto.head()

Check to see if `pandas` can detect the missing values.

Replace the above missing values with `na.nan` and check if the missing values now show.

Now show any rows in the data that have any missing values. HINT: you will need to change `axis = 1`.

### End of exercise

Let's look at some summary statistics for `acceleration`. One quick and easy way to do that is using `describe`.

In [ ]:
auto['acceleration'].describe()

Looking at percentiles helps us get a feel for the distribution of the data. For example, looking at very low or very high percentiles can help us identify **outliers** or **extreme values**. We can use the `quantile` method to get any percentile we want. Here's how:

In [ ]:
percentiles = [0, 0.05, 0.50, 0.95, 1]
auto.quantile(q = percentiles)

Now let's see how we can obtain these same summary statistics, but grouped by `cylinders`. There is already one way we know how to do that: we can first subset the data by `cylinders == 3` and run the summary statistic, then do the same for `cylinders == 4`, etc.

In [ ]:
auto.loc[auto['cylinders'] == 3, :].quantile(q = percentiles)

But of course the above way is tedious. Instead, we can use the `groupby` method to do it all at once. For example, here's the average of each column grouped by `cylinders`.

In [ ]:
auto.groupby('cylinders').mean()

A slightly confusing thing happens if we use a method like `describe` or `quantile` instead of `mean`:

In [ ]:
auto.groupby('cylinders').describe()

The problem with the above table is that there's too much information packed in it. Each value of `cylinders` has its own row, which is fine. But the columns look like they have a hierarchical structure, which makes sense because we asked for many summary statistics for each column.

So what if for example we wanted to store all the results from the above table, and then look at the results for `acceleration` only. Turns out we can do the same thing we usually do:

In [ ]:
results_all = auto.groupby('cylinders').describe()
results_all['acceleration']

The difference is that `results_all['acceleration']` isn't just a single column, but many columns. That's because `results_all` has hierarchical columns. So if we want to drill further down to a specific column, such as `std`, we need to go one level deeper.

In [ ]:
results_all['acceleration']['std']

So far it looks like hierarchical columns are not that complicated. However, things do get a little confusing when we want to cross the hierarchy the other way around. For example, let's say we want to quickly compare the standard deviations of all our columns, grouped by `cylinders`. This information is in the above table, but how do we extract it?

It turns out that to do this, we need to use the `IndexSlice` function in `pandas`. Here how it works:
- we use `loc` to say which rows and columns we want
- we use `slice(None)` to say we want everything at a given hierarchy
- if we want to filter at any level of hierarchy, we provide the index we want to filter by

In [ ]:
from pandas import IndexSlice as idx
results_all.loc[slice(None) , idx[slice(None), 'std']]

By the way, the same way that we can have hierarchical columns, we can also have hierarchical rows, more specifically hierachical row indexes. Examine the result generated by the cell below. Do see the hierarchical structure of the row index?

In [ ]:
auto.groupby('cylinders').quantile(q = percentiles)

Hierarchical indexes are called `MultiIndex` in `pandas`. This is a more advanced topic and we will leave it at that for now.

### Exercise

We saw how a histogram shows us counts for **evenly-sized** intervals of a numeric column. What if we wanted to see counts for any intervals we specify, evenly-sized or not? Here's an example:

Find out how many cars have `mpg` less than 18, between 18 and 25, and 25 or more?

Note that there are many ways of doing this, so to narrow it down a little do this:
- use `loc` to limit data to the above ranges
- use the `shape` method to get row counts

Check what happens if some of your data points fall outside of any of intervals. Does this match what you would expect?

### End of exercise

There is another, much faster way to solve the exercise above: by using the `pd.cut` function and specify the cut-offs using the `bins` argument.

In [ ]:
auto['mpg_cat'] = pd.cut(auto['mpg'], bins = [0, 18, 25, 100])
auto[['mpg', 'mpg_cat']].head()

Now we can just use `value_counts` to get our counts.

In [ ]:
auto['mpg_cat'].value_counts()

### Exercise

Let's create `mpg_cat` again, but make the following changes to it: 

- Note that the choice of using 0 and 100 as the lower and upper bounds is a little arbitrary. So use `-np.Inf` and `np.Inf` instead.
- By default, cut labels the catgories based on the interval range it covers, but we can change the labels to anything we like using the `labels` argument. Rename the labels to `low`, `med` and `high`.

### End of exercise

The `pd.cut` function is different from the other functions (or methods) we learned about so far. The `describe`, `mean`, `quantile`, or `value_counts` functions are **feature summarization** functions, but `pd.cut` is a **feature transformation** function, meaning that not only its input is a feature, but its output is also a feature. In the above example, the input to `pd.cut` was `mpg`, and the output was the `mpg_cat` which we appended to the data as a new column.

There are many other **feature transformation** functions. In fact, **feature engineering** consists of running these feature transformations on the data and gradually modifying existing columns and adding new features to the data, with the goal of ending up with features that are more useful to the model than the original features we started with.

### Exercise

Normalize `mpg`, `displacement`, `weight` and `acceleration`. Instead of overwriting the original columns, add the normalized features as new columns and name each column using the column name and a `_norm` suffix. You are free to choose between Z-normalization or min-max normalization.

Unless your Python skills are improving by leaps and bounds, you probably normalized the features one at a time. What if we wanted to do it all at once? Write a loop to iterate over the four columns and normalize each. To make it easier, we already put the column names in a list for you.

In [ ]:
num_cols = ['mpg', 'displacement', 'weight', 'acceleration']

There is an even better way to run our transformations all at once without writing a loop. First we have to write a function whose input is an array and whose output is an array of the same size with the values normalized. Write such a function and use the below cell to test it and make sure it worked.

In [ ]:
def normalize(x):
    x_norm = x # write your function
    return x_norm

x_test = np.array([3, 5, 9, 11, 2, 0])
normalize(x_test)

Time to apply the function to the data. To apply the function, we use the `apply` method.
- note that we need to limit the data to only the four columns we wish to transform
- we need to use the `axis = 1` argument to let `apply` know that the transformation applies to columns (`axis = 0` would apply it to rows, which is not what we want here)

Apply the transformation to the data and check the results using `head`. Then do an additional sanity check:
- if your function is doing Z-normalization, then check the mean and standard deviation of your normalized columns to make sure they are 0 and 1 respectively
- if your function is doing min-max normalization, then check the minimum and maximum values of your normalized columns to make sure they are 0 and 1 respectively

### End of exercise

Since normalization is a very common task, you shoudn't be surprised to find out that there is already a function for doing this. I hope you still found the previous exercise useful!

However, this time we have to go to the `sklearn` library to find our function. The `sklearn` library is Python's most common machine learning library and one that we will return to in future lectures. In addition to the machine learning algorithms we will learn about, `sklearn` also has functions for **pre-processing data**, which is a vague term that includes tasks such as missing-value imputation, feature engineering and so on. Let's see how we can use it to normalize our data.

There's one important thing to note about `sklearn` before we continue: We just saw how we can use `pandas` to do data pre-processing and feature engineering, now we're going to see how we can use `sklearn`. When using `pandas` transformation on a `DataFrame` return a `DataFrame` and everything is relatively easy. But with `sklearn` transformations don't return nice-looking `DataFrame` objects. Instead they return `numpy` arrays, so it's up to us to convert the arrays back into `DataFrame` if we need to. In **production** we usually wouldn't need to do that, since we can pass the arrays directly to the ML algorithm for training and for scoring. But during **development** it can help us trace our steps and see intermediate results as we work toward the final product.

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
num_cols_norm = [c + '_norm' for c in num_cols] # names of transformed columns

At first blush, this code might look a little strange, but this pattern as we will see is very common to ML-related tasks in `sklearn`:
- initialize the process by choosing the function (with arguments we wish, if any)
- run `fit` first on the data, followed by `transform` to apply the transformation

In [ ]:
minmax_scaler = MinMaxScaler()
minmax_scaler.fit(auto[num_cols])
auto[num_cols_norm] = pd.DataFrame(minmax_scaler.transform(auto[num_cols]))
auto[num_cols_norm].head()

In [ ]:
sns.pairplot(auto[num_cols_norm]);

Here's the same example, but using Z-normalization.

In [ ]:
znorm_scaler = StandardScaler()
znorm_scaler.fit(auto[num_cols])
auto[num_cols_norm] = pd.DataFrame(znorm_scaler.transform(auto[num_cols]))
auto[num_cols_norm].head()

Let's look at the scatter plot matirx for the normalized features.

In [ ]:
sns.pairplot(auto[num_cols_norm]);

In either case, it doesn't look like normalization did anything to make our scatter plot matrix looks different. Do you notice what changed? The answer is that the **scale of the data** is what changed. Just check the $x$ and $y$ axes and you'll see. Normalization is not really supposed to change the nature of the data, just put all features on the same scale.

Let's look one last time at the code for normalizing the data using `sklearn`.

In [ ]:
minmax_scaler = MinMaxScaler()
minmax_scaler.fit(auto[num_cols])
auto[num_cols_norm] = pd.DataFrame(minmax_scaler.transform(auto[num_cols]))
auto[num_cols_norm].head()

There's two things we need to point out about the code above:

In the third line, notice how we used `pd.DataFrame` to turn the data into a `DataFrame` object. This is because `sklearn` returns a `numpy` array instead of a `DataFrame`. As we saw earlier, the distinction is most of the time unimportant since a `DataFrame` is just a sort of wrapper for a `numpy` array, but if we want to use `DataFrame` methods like `head` to examine the results, we need to do our own conversion.

You might be curious why we use `fit` followed by `transform`. What exactly happens when we run `fit`? Why should those two not be steps not be a single step? The answer to that will become obvious later in the context of machine learning, but here's a short answer using `MinMaxScaler` as our example:
  - When we run `fit` we find the min and max for the columns and rememeber it.
  - When we run `transform` we apply the transformation using the min and max we found when we ran `fit`.

This means that we can learn the min and max once, and then apply the **same** transformation (with the same min and max) not just to the original data, but any future data. In machine learning, this has important consequences, but that's the topic of a future lecture.

It's time now to shift focus away from numeric features and talk about feature engineering for categorical features. We will use another dataset for this.

In [ ]:
tennis = pd.read_csv("data/tennis.csv")
tennis.head()

As we can see, the data only contains categorical features. Let's see what the categories are for each feature.

### Exercise

Write a loop to obtain counts of unique values for each column in the data. We can use the `value_counts` method to get counts for a column.

### End of exercise

Let's see what happens if we one-hot-encode the above data. Just like normalization, one-hot-encoding is a common pre-processing task and we can turn to the `sklearn` library to do the hard part for us. And once again, we will see the pattern of using `fit` followed by `transform`.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

onehot = OneHotEncoder(sparse = False) # initialize one-hot-encoder
onehot.fit(tennis)
col_names = onehot.get_feature_names(tennis.columns) # this allows us to properly name columns
tennis_onehot =  pd.DataFrame(onehot.transform(tennis), columns = col_names)
tennis_onehot.head()

So we can see that one-hot encoding created a **binary feature** for **each category of each categorical column** in the data. These binary features are also sometimes called **dummy variables**. Why do we do this? The reason we do this is that most machine learning algorithms do not work **directly** with categorical data, so we need to encode the categorical data which turns it into numeric data. One-hot encoding is just one type of encoding, but it is the most common one.

One last note about the `sklearn` pre-processing transformations we learned about in this notebook: If you look at examples online, you may notice that instead of calling `fit` and `transform` separately, you can call `fit_transform` which combines the two steps into one. This may seem reasonable and saves you one extra line of code, but we discourage it. The following exercise will illustrate why, but the main reason will become clear when we talk about machine learning.

### Exercise

Let's return to the tennis data, and fit a one-hot encoder on it.

In [ ]:
tennis = pd.read_csv("data/tennis.csv")
onehot = OneHotEncoder(sparse = False) # initialize one-hot-encoder
onehot.fit(tennis)

We now introduce a change: we will add an additional category to `outlook` called `amazing`. The important point here is that we introduce this additional category **after** we ran `fit` on the one-hot encoder above.

In [ ]:
tennis.loc[2, "outlook"] = "amazing" # introduce a category unseen when we ran fit

Now let's see what happens if we try to run `transform` on the data to one-hot encode the features. If you run the code below you'll notice that we get an error. What is the error for?

In [ ]:
col_names = onehot.get_feature_names(tennis.columns)
tennis_onehot =  pd.DataFrame(onehot.transform(tennis), columns = col_names)
tennis_onehot.head()

Is it a good thing that we got an error? The answer is it depends: 

- If we are okay with letting new categories slip through, we can return to where we initiated `OneHotEncoder` and change the `handle_unknown = 'ignore'` (default value is `'error'`). Make this change and rerun the code. What is the one-hot encoded value for `outlook` at the row that we changed?
- If you want to make sure that we preserve **data integrity** so that the data we call `transform` on matches the schema of the data we ran `fit` on, then we want errors like this to stop us in our tracks so we have a change to see why the data changed.

### End of exercise

Now that we covered common transformations for both numeric and categorical features, let's look at an example of a special transformation very common in the retail industry: RFM or recency-frequency-monetary transformation.

Prior to running the following code, make sure you upgrade `pandas` to version `1.0.1` by running `pip install pandas==1.0.1`. You can check the current version of pandas by running the next cell.

In [ ]:
import pandas as pd
pd.__version__

Let's read our data.

In [ ]:
col_names = ['user_id', 'gender', 'address', 'store_id', 'trans_id', 'timestamp', 'item_id', 'quantity', 'dollar']
churn = pd.read_csv("data/retail-churn.csv", sep = ",", skiprows = 1, names = col_names)
churn.head()

The first thing we need to do is convert the datetime column to `datetime`.

In [ ]:
churn['timestamp'] = pd.to_datetime(churn['timestamp'], format = '%m/%d/%Y %H:%M')
churn['date'] = churn['timestamp'].dt.date

Notice that we run into a problem with our data: the **granularity** of the data is not daily spend, but rather individual transactions. You can see that because the same user has multiple transactions with the same timestamp. This means that before we can do feature engineering, we need to **aggregate** the data to daily data so that `user_id` and `date` are unique for each row. In the process, we also get rid of all the columns we don't need.

In [ ]:
churn_agg = churn.groupby(['user_id', 'date']).agg({'dollar': 'sum', 'quantity': 'sum'})
churn_agg = churn_agg.reset_index()

In [ ]:
# churn_agg['date'] = churn_agg['date'].astype('datetime64')
churn_agg['date'] = pd.to_datetime(churn_agg['date'])
churn_agg.dtypes

With the aggregated data, we can now obtain recency, frequency and monetary features:
- In `pandas` recency is a kind of **difference** feature, because it's based on calculating the difference between the current date and a previous date (called a **lag**). We can use the `diff` function to get recency.
- Frequency and monetary features are called **rolling** features, because it is a type of cumulative sum but over a moving window. We can use the `rolling` function to get frequency and monetary, where `window = '7D'` means we're using a 7-day moving window, and `on = 'date'` means that the moving window moves over `date`.

In [ ]:
roll_sum = churn_agg.groupby('user_id').rolling(window = '7D', on = 'date')['dollar'].sum()
roll_cnt = churn_agg.groupby('user_id').rolling(window = '7D', on = 'date')['quantity'].sum()
roll_min = churn_agg.groupby('user_id')['date'].diff(periods = 1)

We now combine all three features into a single `DataFrame` which we call `churn_roll`. For recency, we can use `fillna` to replace missing values with a large value like 100 days or whatever makes business sense.

In [ ]:
churn_roll = pd.concat([roll_sum, roll_cnt], axis = 1)
churn_roll.columns = ['dollar_roll_sum_7D', 'quantity_roll_sum_7D']
churn_roll = churn_roll.reset_index()
churn_roll['last_visit_ndays'] = roll_min.fillna(pd.Timedelta('100 days'))

To see if things worked, let's first comibne the aggregated data with the data with the new features. We can use the `merge` method to do this. We can check the recency (named `last_visit_ndays`), frequency (named `quantity_roll_sum_7D`) and monetary (named `dollar_roll_sum_7D`) columns to make sure the calculations look correct.

In [ ]:
churn_all = churn_agg.merge(churn_roll, how = "inner", on = ['user_id', 'date'], 
                            validate = 'one_to_one')
churn_all.head()

One take-away from the above example is that feature engineering can be a complicated topic, and relies to some extent on creativity and domain knowledge, as we saw with time series data and RFM. For this reason, some modern machine learning libraries are working on what is called **automated feature engineering** to see if algorithms can automatically figure out a set of good features to use by the machine learning model.